In [1]:
import chromadb
from chromadb.config import Settings as StChroma
import json
from sqlalchemy import create_engine, text
from llama_index.core.node_parser. import (
  
)


In [13]:
engine = create_engine('sqlite:///data/out/recipes.db')

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM recipes"))
    print(result.)
    keys = result.keys()
    data_sql = [dict(zip(keys, row)) for row in result]


document_sql = [{"id": str(i), "document": str(item)} for i, item in enumerate(data_sql)]


In [14]:
ids = [str(i) for i in range(len(document_sql))]
document_sql = [json.dumps(item) for item in document_sql]
document_sql[0]


'{"id": "0", "document": "{\'category\': \'Bebidas\', \'name\': \'Margarita Cristalina\', \'source\': \'https://www.kiwilimon.com/receta/bebidas/cocteles/margarita-cristalina\', \'prep_time\': \'5 mins\', \'cook_time\': \'N/A\', \'difficulty\': \'Baja\', \'ingredients\': \'1 1/2 onzas de Tequila Patr\\u00f3n Cristalino\\\\n1/2 onzas de jugo de lim\\u00f3n\\\\n1/4 onzas de miel de agave\\\\n2 golpes de soluci\\u00f3n salina\\\\nsuficiente de hielo\\\\nlim\\u00f3n amarillo al gusto\', \'preparation\': \'Combina el Tequila Patr\\u00f3n Cristalino, el jugo de lim\\u00f3n, la miel de agave y la soluci\\u00f3n salina en un shaker.\\\\nAgrega los hielos y agita unos segundos.\\\\nSirve la margarita cristalina en un vaso old fashioned y decora con lim\\u00f3n amarillo caramelizado.\'}"}'

In [5]:
print(f"len of documents:{len(document_sql)}\nlen of ids:{len(ids)}")


len of documents:1397
len of ids:1397


In [ ]:
client = chromadb.Client(settings=StChroma(allow_reset=True))
client.reset()
collection = client.create_collection("recipes")


collection.add(documents=document_sql, ids=ids)


In [ ]:
print(json.dumps(collection.get(limit=1), indent=4))


In [ ]:
results = collection.query(
    query_texts=["give me 5 recipes with chocolate"], n_results=5
)
print(json.dumps(results, indent=2))


### Vectors with Chroma


In [19]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    StorageContext,
    SimpleDirectoryReader,
)


In [35]:
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.5, max_tokens=100)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
# embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
Settings.embed_model = embed_model


In [42]:
# chroma


db = chromadb.PersistentClient(path="./chroma_recipe", settings=StChroma(allow_reset=True))
# db.reset()
chroma_collection = db.get_or_create_collection(name="recipes")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
documents = SimpleDirectoryReader("data/loader").load_data()
splitter = 

index = VectorStoreIndex.from_documents(
  documents=documents,
  storage_context=storage_context,
  embed_model=embed_model,
  transformations=
)
# chroma_collection.add(documents=document_sql, ids=ids)
print(chroma_collection.get(limit=1))


InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 384

In [30]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [34]:
chroma_collection.query(query_texts=["What is the collection length"], n_results=1)


{'ids': [['1146']],
 'distances': [[1.6897680759429932]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [['{"id": "1146", "document": "{\'category\': \'Recetas A La Parrilla\', \'name\': \'Costillas De Cerdo Al Tequila\', \'source\': \'https://www.kiwilimon.com/receta/recetas-a-la-parrilla/costillas-de-cerdo-al-tequila\', \'prep_time\': \'20 mins\', \'cook_time\': \'3h\', \'difficulty\': \'Baja\', \'ingredients\': \'1 cucharada de sal, ahumada, para el rub\\\\n2 cucharadas de polvo de ajo, para el rub\\\\n2 cucharadas de cebolla en polvo, para el rub\\\\n1 cucharada de pimienta gorda, molida, para el rub\\\\n13 chiles cascabel, asado y molido, para el rub\\\\n2 cucharadas de pimienta cayena, para el rub\\\\n1 cucharada de comino, para el rub\\\\n1 cucharada de or\\u00e9gano, para el rub\\\\n5 cucharadas de ajo, en hojuela, para el rub\\\\n3 cucharadas de mostaza\\\\n1 costillar de cerdo, 2-3 kg, para la marinada\\\\n1 taza de az\\u00facar mascabado\\\\n1/2 tazas de mantequ

In [31]:
query_engine = index.as_query_engine()
response = query_engine.query("Tell me 5 recipes with chocolate")
print(response)


InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 384